In [3]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import LabelEncoder, RobustScaler

In [4]:
pwd

'/Users/yogi/Desktop/Data Science and Analytics/HackerEarth/FutureHackathon'

In [22]:
## load data
products = pd.read_csv('./Q1ClData/cproducts.csv')
payment = pd.read_csv('./Q1ClData/ctender.csv')

products = products.head(2000)
payment = payment.head(2000)

In [7]:
products.head()

,customerID,DOB,Gender,State,PinCode,transactionDate,store_code,store_description,till_no,transaction_number_by_till,promo_code,promotion_description,product_code,product_description,sale_price_after_promo,discountUsed
0,BBID_20482,1975-10-20,male,MADHYA PRADESH,453441.0,2016-05-01,2655,BB-INDORE-MALHAR MEGA MALL,2,85246,NONPROMO,NaN,1000446431,PATANJALI CHOCOS 125g,55.00,Payback
1,BBID_20485,1955-07-21,female,NaN,999999.0,2017-03-09,2655,BB-INDORE-MALHAR MEGA MALL,8,78829,NONPROMO,NaN,1000010431,SOO FRESH ROLLS HOT DOG 4P,30.00,Payback
2,BBID_20498,1981-03-19,female,MAHARASHTRA,400064.0,2015-08-12,2655,BB-INDORE-MALHAR MEGA MALL,13,1064,0000971754,AD 399 FLAT LADIES JEANS FASHION EXC,1000077851005,"DLJ-0010-BSF-2-CJ-FL, 34, BLACK",418.95,Payback
3,BBID_20499,1986-05-10,male,MAHARASHTRA,400099.0,2016-12-18,2615,BB-AMRITSAR-TRILIUM MALL,14,51080,NONPROMO,NaN,1000443231,PATANJALI TOMATO KETCHUP BT 500g,70.00,Payback
4,BBID_204110,1988-02-29,male,MADHYA PRADESH,474001.0,2015-12-31,2655,BB-INDORE-MALHAR MEGA MALL,9,21911,0000974126,NMP 5% OFF CONCEPT DISCOUNT 2655,300481741,MAGGI NDL MERI MASALA PP 70g,34.20,Payback


In [8]:
products.shape

(11972, 16)

In [9]:
payment.head()

,customerID,DOB,Gender,State,PinCode,transactionDate,store_code,store_description,till_no,tender_type,transaction_number_by_till,payment_amount_by_tender,PaymentUsed
0,BBID_2044,1984-04-02,male,NaN,626125.0,2015-09-03,4986,BB-MADURAI-KOCHADAI VILLAGE,22,GVOWN,9208,1000.00,GiftVoucher
1,BBID_2044,1984-04-02,male,NaN,626125.0,2015-09-03,4986,BB-MADURAI-KOCHADAI VILLAGE,22,CASH,9208,10.46,CASH
2,BBID_2044,1984-04-02,male,NaN,626125.0,2016-04-17,4986,BB-MADURAI-KOCHADAI VILLAGE,19,EDCAXIS,90269,677.28,AXIS
3,BBID_2044,1984-04-02,male,NaN,626125.0,2016-04-17,4986,BB-MADURAI-KOCHADAI VILLAGE,19,MOBI,90268,1042.00,One MobiKwik Systems
4,BBID_2044,1984-04-02,male,NaN,626125.0,2016-04-17,4986,BB-MADURAI-KOCHADAI VILLAGE,19,MOBI,90269,1000.00,One MobiKwik Systems


In [10]:
payment.shape

(149752, 13)

In [23]:
#Product Missing Data
#Missing Data
total = products.isnull().sum().sort_values(ascending=False)
percent = (products.isnull().sum()/products.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(20)

,Total,Percent
promotion_description,1535,0.7675
State,409,0.2045
PinCode,73,0.0365
Gender,45,0.0225
discountUsed,0,0.0000
sale_price_after_promo,0,0.0000
product_description,0,0.0000
product_code,0,0.0000
promo_code,0,0.0000
transaction_number_by_till,0,0.0000


In [24]:
#Payment Missing Data
total = payment.isnull().sum().sort_values(ascending=False)
percent = (payment.isnull().sum()/payment.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(20)

,Total,Percent
State,273,0.1365
PaymentUsed,18,0.0090
payment_amount_by_tender,0,0.0000
transaction_number_by_till,0,0.0000
tender_type,0,0.0000
till_no,0,0.0000
store_description,0,0.0000
store_code,0,0.0000
transactionDate,0,0.0000
PinCode,0,0.0000


In [25]:
## fill missing values

products['promotion_description'].fillna('no_promo', inplace=True)
products['Gender'].fillna('no_gender', inplace=True)
products['State'].fillna('no_state', inplace=True)
products['PinCode'].fillna(-1, inplace=True)
products['DOB'].fillna("1", inplace=True)

In [26]:
## convert data into numeric / float

for c in products.columns:
    lbl = LabelEncoder()
    if products[c].dtype == 'object' and c not in ['store_description','customerID','transactionDate']:
        products[c] = lbl.fit_transform(products[c])

In [27]:
## scaling, creating matrix and running k-means

stores = list(set(products['store_code']))
print(stores)

cluster_labels = []
cluster_store = []
cluster_data = []
cluster_customers = []
cluster_score = []

for x in stores:
    cld = products[products['store_code'] == x]
    cluster_customers.append(cld['customerID'])
    cld.drop(['store_code','customerID','transactionDate','store_description'], axis=1, inplace=True)
    
    print(cld.shape)
    print(cld.head(1))
    
    rbs = RobustScaler()
    cld2 = rbs.fit_transform(cld)
    print(cld2[0])
    
    km1 = KMeans(n_clusters=3)
    km2 = km1.fit(cld2)
    label = km2.predict(cld2)
    
    s_score = silhouette_score(cld2, label)
    cluster_score.append(s_score)
    print(s_score)
    
    cluster_labels.append(label)
    cluster_store.append(np.repeat(x, cld.shape[0]))
    cluster_data.append(cld2)

[3692, 2615, 2906, 4986, 2655]
(47, 12)
   DOB  Gender  State   PinCode  till_no  transaction_number_by_till  \
8  759       0      7  122002.0        5                       11873   

   promo_code  promotion_description  product_code  product_description  \
8         403                    384    1000257325                  119   

   sale_price_after_promo  discountUsed  
8                  113.44             0  
[-0.44833625 -2.         -2.5        -0.29235313  0.         -0.58656693
  0.          0.          0.99886506 -1.04610951  0.58665085  0.        ]
0.83949361211
(42, 12)
    DOB  Gender  State   PinCode  till_no  transaction_number_by_till  \
3  1209       1     15  400099.0       14                       51080   

   promo_code  promotion_description  product_code  product_description  \
3         403                    384    1000443231                 1038   

   sale_price_after_promo  discountUsed  
3                    70.0             0  
[ 0.31204188  0.         -0.

/Users/yogi/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


(441, 12)
   DOB  Gender  State   PinCode  till_no  transaction_number_by_till  \
6  739       1     15  401105.0       11                       33822   

   promo_code  promotion_description   product_code  product_description  \
6         217                     20  1000479429005                 1244   

   sale_price_after_promo  discountUsed  
6                 1503.75             0  
[ -2.00239808e-01   0.00000000e+00   1.00000000e-01  -1.01792000e+01
   5.71428571e-01  -3.84944397e-02  -1.86000000e+02  -3.64000000e+02
   1.12107918e+03   7.56130790e-01   1.20562500e+01   0.00000000e+00]
0.888854710764
(620, 12)
    DOB  Gender  State   PinCode  till_no  transaction_number_by_till  \
9  1210       1     20  625011.0       13                       79027   

   promo_code  promotion_description  product_code  product_description  \
9         403                    384     108014378                 1058   

   sale_price_after_promo  discountUsed  
9                    72.0          

In [16]:
# check mean score per store
np.mean(cluster_score)

0.92850344428428089

In [17]:
## merge list into ndarray
cluster_data = np.concatenate(cluster_data)

## check if the array has same rows as products file - Yes!
cluster_data.shape

(11972, 12)

In [18]:
## convert nested lists as 1d array
cluster_customers = np.concatenate(cluster_customers)
cluster_store = np.concatenate(cluster_store)
cluster_labels = np.concatenate(cluster_labels)

## create submission files
sub1 = pd.DataFrame({'customerID':cluster_customers, 'store_code':cluster_store, 'cluster':cluster_labels})

np.savetxt('../subOne_18.txt', cluster_data)
sub1.to_csv('../subtwo_18.csv', index=False)